In [1]:
import numpy as np
import pandas as pd
import shap

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml import PipelineModel
from shapash.explainer.smart_explainer import SmartExplainer
import pyspark
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import os


In [3]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName("MyApp") \
            .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.10.0") \
            .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
            .getOrCreate()
import synapse.ml

In [4]:
df = spark.read.csv('../MODEL_BUILDING/processed_parking.csv',header=True,inferSchema=True)

In [5]:
model = PipelineModel.load('../MODELS/PipelineModelRF')

In [ ]:
from mmlspark import *
from mmlspark import FluentAPI   


In [7]:
explain_instances = (
    model.transform(df).limit(5).repartition(200).cache() 
)
display(explain_instances)

DataFrame[Registration_State: string, Plate_Type: string, Violation_Code: int, Vehicle_Body_Type: string, Vehicle_Make: string, Issuing_Agency: string, Street_Code1: int, Street_Code2: int, Street_Code3: int, Violation_Location: int, Issuer_Precinct: int, Issuer_Command: string, Violation_County: string, Violation_In_Front_Of_Or_Opposite: string, Month: double, Day: double, Meridiem: string, Time_Hour: double, Street_Code2_index: double, Vehicle_Make_index: double, Issuer_Precinct_index: double, Registration_State_index: double, Violation_In_Front_Of_Or_Opposite_index: double, Meridiem_index: double, Issuer_Command_index: double, Street_Code1_index: double, Violation_Code_index: double, Street_Code3_index: double, Violation_Location_index: double, Issuing_Agency_index: double, Vehicle_Body_Type_index: double, Plate_Type_index: double, label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [11]:
df.count()

8142488

In [8]:
from pyspark.sql.functions import broadcast
from synapse.ml.explainers import *
shap = TabularSHAP(
    inputCols = df.columns,
    outputCol = 'shapValues',
    numSamples=50000,
    model=model,
    targetCol='probability',
    targetClasses=[1],
    backgroundData=broadcast(train.limit(100).cache())
)
shap_df = shap.transform(explain_instances)

NameError: name 'train' is not defined

In [ ]:
vec_access = udf(lambda v, i: float(v[i]), FloatType())
vec2array = udf(lambda vec: vec.toArray().tolist(), ArrayType(FloatType()))
shaps = (
    shap_df.withColumn("probability", vec_access(col("probability"), lit(1)))
    .withColumn("shapValues", vec2array(col("shapValues").getItem(0)))
    .select(
        ["shapValues", "probability", "label"] + df.columns
    )
)



In [ ]:
shaps.columns

In [ ]:
shaps_local = shaps.toPandas()
shaps_local.sort_values("probability", ascending=False, inplace=True, ignore_index=True)
pd.set_option("display.max_colwidth", None)
shaps_local